In [1]:
# !gdown 1mKLyBFIoKKWvCihQiZ6cjqx8TKYHRWFA
# !gdown 1NMH2EMoK6roGUGZnpM1WByLMYuPw6Gzp
# !gdown 1tCZuz2AjPTp5ok5avZAXAAkCap8GmeIL

In [2]:
# !pip install datasets
# !pip install seqeval
# !pip install transformers[torch]

In [3]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification


/home/sivakorn/forest_type/img_reg/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-14 00:06:38.180640: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 00:06:38.204109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 00:06:38.238225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 00:06

In [4]:
def load_data(fname,tag_id=None):
  data = []
  if tag_id is None:
    tag_id = {}
  with open(fname) as fp:
      lines = fp.readlines()
      tmp_sen = []
      for line in lines:

          if line.strip()=="":
              if len(tmp_sen) > 0:
                  tokens = []
                  tags = []
                  for token,tag in tmp_sen:
                      tokens.append(token)
                      #if tag_id is None:
                      if tag not in tag_id.keys():
                        #print(tag)
                        tag_id[tag] = len(tag_id)

                      #print(tag_id[tag])
                      tags.append(tag_id[tag])



                  data.append({"tokens":tokens,"tags":tags})
                  tmp_sen = []

              continue

          _x = line.split('\t')

          tmp_sen.append((_x[0],_x[1].strip()))
  return data,tag_id

In [5]:
import json

def write_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json_line = json.dumps(entry)
            file.write(json_line + '\n')

In [6]:
train_data,tag_id = load_data('train.txt')



# test_data,_ = load_data('test.csv',tag_id)



In [7]:
tag_id

{'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

In [8]:
write_jsonl(train_data,'train.json')



In [9]:
task = "asp"
model_checkpoint = "thanaphatt1/WangchanBERTa-LST20"
batch_size = 32

file_dict = {
  "train" : "./train.json",
  # "test" : "./test.json"
}

datasets =  load_dataset(
  'json',
  data_files=file_dict
)

Generating train split: 609 examples [00:00, 28464.00 examples/s]


In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 609
    })
})

In [11]:
from datasets import load_dataset, DatasetDict

datasets = datasets['train'].train_test_split(test_size=0.1)  # 10% for validation

# # Rename the splits to train and validation
datasets['val'] = datasets.pop('test')



In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 548
    })
    val: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 61
    })
})

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
print(tokenizer("ผมเป็นคนหล่อมาก"))
print(tokenizer(["ผม","เป็น","คนหล่อ","มาก"]))

{'input_ids': [0, 3337, 1039, 3712, 5373, 17277, 47, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[0, 76, 2], [0, 12, 2], [0, 5373, 1452, 2], [0, 47, 2]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1]]}


In [15]:

def tokenize_and_align_labels(examples):
    print(type(examples["tokens"]), examples["tokens"])
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    label_all_tokens = True
    #print('tokenized_inputs:',tokenized_inputs)
    labels = []
    lab2index = {}
    word_idxs = []
    for i, label in enumerate(examples[f"tags"]):
        #print(label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        #print('x word_ids',word_ids)

        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)
        word_idxs.append(word_ids)
    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_idxs
    return tokenized_inputs

In [16]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

<class 'list'> [['สินค้า', 'ที่', 'ได้รับ', 'ไม่', 'ตรง', 'ตามที่', 'สั่งอาหาร', 'แมว', 'ที่', 'ได้', 'ไม่', 'ใช่', 'ยี่ห้อ', 'ที่', 'สั่ง', 'สั่งอาหาร', 'เม็ด', 'อาหาร', 'เปียก', 'แพ็ค', 'สินค้า', 'มา', 'ไม่', 'ดี', 'เลย', 'ไร้', 'คุณภาพ', 'สุด', 'สุด', 'ถุง', 'ฉีกขาด', 'ทำให้', 'สินค้า', 'ได้รับ'], ['ดู', 'จาก', 'ภาพ', 'ใน', 'เพจ', 'คิด', 'ว่า', 'ชุดนอน', 'ที่', 'ต้อง', 'การสั่งซื้อ', 'คงจะ', 'น่ารัก', 'และ', 'สวย', 'มาก', 'ๆ', 'และ', 'ต้อง', 'ใส่', 'สบาย', 'ด้วย', 'ที่ไหนได้', 'พอ', 'ได้รับ', 'สินค้า', 'จริงๆ', 'โอ้โห', 'คนละเรื่อง', 'เลย', 'เนื้อผ้า', 'อย่าง', 'หยาบ', 'ลาย', 'สกรีน', 'ก็', 'ทำ', 'แบบ', 'หยาบ', 'ๆ', 'เสียดาย', 'เงิน', 'มาก', 'เที่ยว', 'หน้า', 'ไม่', 'สั่ง', 'แล้ว', 'ร้าน', 'นี้', 'เข็ด'], ['หูฟัง', 'บลูทูธ', 'ระดับ', 'พรีเมียม', 'ที่มา', 'พร้อมกับ', 'เทคโนโลยี', 'การ', 'ลด', 'เสียงรบกวน', 'แบบ', 'อัจฉริยะ', 'ที่', 'นำเสนอ', 'ประสบการณ์', 'การ', 'ฟัง', 'ที่', 'เงียบสงบ', 'และ', 'ยอดเยี่ยม', 'มี', 'คุณภาพ', 'เสียง', 'ที่', 'น่าทึ่ง', 'และ', 'สามารถ', 'ปรับ', 'แต่ง', '

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

<class 'list'> [['สินค้า', 'มือหนึ่ง', 'ตรง', 'ตาม', 'แบบ', 'ที่', 'สั่ง', 'ห่อ', 'มา', 'ดี', 'มิดชิด', 'มี', 'ประกัน', 'หลัง', 'หลัง', 'ขาย', 'ใช้งาน', 'ได้ดี', 'เปิด', 'เครื่อง', 'ไม่', 'มี', 'สะดุด', 'หน้าจอ', 'ใหญ่', 'กว่า', 'แบบ', 'ที่', 'สั่ง', 'ภาพ', 'สี', 'คมชัด', 'มาก', 'แต่', 'แบตเตอรี', 'ชาร์จ', 'ไฟ', 'ค่อนข้าง', 'ช้า'], ['ได้รับ', 'สินค้า', 'ตรง', 'ตามที่', 'สั่ง', 'สินค้า', 'ตรง', 'ปก', 'แข็งแรง', 'สี', 'สวย', 'ไม่', 'มี', 'รอย', 'บุบ', 'เลย', 'แต่', 'ห่อ', 'มาก', 'ดีมาก', 'น่าจะ', 'เกิด', 'จาก', 'ขนส่ง', 'ราคา', 'ดี', 'สุด', 'ๆ', 'ซื้อ', 'เถอะ', 'คุ้ม', 'มาก', 'แม่', 'ตำ', 'ด่วน', 'ปรับ', 'ไฟ', 'ได้', 'หลาย', 'ระดับ'], ['สินค้า', 'ที่', 'มาจาก', 'ต่างประเทศ', 'พอ', 'ผ่าน', 'มาถึง', 'ไทย', 'ก็', 'ต้อง', 'ให้', 'ร้านค้า', 'เป็น', 'คน', 'ดำเนินการ', 'เรื่อง', 'สินค้า', 'ให้', 'กับ', 'ทางไปรษณีย์', 'ขนส่ง', 'ซึ่ง', 'ต้อง', 'มี', 'การ', 'ตรวจสอบ', 'ความ', 'เรียบร้อย', 'ระดับ', 'หนึ่ง', 'ถึง', 'ต้อง', 'รอ', 'เป็น', 'อาทิตย์', 'ก็', 'คุ้มค่า', 'กับ'], ['ร้านค้า', 'ตอบ', 'แชท', '

Map: 100%|██████████| 61/61 [00:00<00:00, 1628.10 examples/s]


In [17]:
39
49
39
49
49

49

In [18]:
#tokenized_datasets['train']['input_ids'][0]
print(tokenized_datasets['train']['tokens'][0])
print(tokenized_datasets['train']['input_ids'][0])
print(tokenized_datasets['train']['tags'][0])
print(tokenized_datasets['train']['labels'][0])
print(tokenized_datasets['train']['word_ids'][0])

['สินค้า', 'ที่', 'ได้รับ', 'ไม่', 'ตรง', 'ตามที่', 'สั่งอาหาร', 'แมว', 'ที่', 'ได้', 'ไม่', 'ใช่', 'ยี่ห้อ', 'ที่', 'สั่ง', 'สั่งอาหาร', 'เม็ด', 'อาหาร', 'เปียก', 'แพ็ค', 'สินค้า', 'มา', 'ไม่', 'ดี', 'เลย', 'ไร้', 'คุณภาพ', 'สุด', 'สุด', 'ถุง', 'ฉีกขาด', 'ทำให้', 'สินค้า', 'ได้รับ']
[0, 2168, 5, 17266, 16, 250, 13133, 5, 18927, 129, 1008, 5, 9, 16, 238, 1600, 5, 383, 18927, 129, 1068, 129, 3168, 2871, 2168, 19, 16, 48, 79, 865, 365, 211, 211, 875, 11641, 11889, 2168, 17266, 2]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
[-100, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, -100]
[None, 0, 1, 2, 3, 4, 5, 5, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, None]


In [19]:
from transformers import RobertaConfig, RobertaForTokenClassification, Trainer, TrainingArguments

config = RobertaConfig.from_pretrained(model_checkpoint)
# config.hidden_dropout_prob = 0.3  # Increase dropout rate
# config.attention_probs_dropout_prob = 0.3  # Increase dropout rate
config.num_labels = len(tag_id)

In [20]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, config=config ,ignore_mismatched_sizes=True )

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at lst-nectec/HoogBERTa-NER-lst20 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([48]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([48, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(74905, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [22]:
from transformers import TrainingArguments

batch_size = 32  # Example batch size, adjust based on your GPU memory

args = TrainingArguments(
    output_dir=f"test-{task}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=500,
    weight_decay=0.01,
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=50,                # Log every 50 steps
    save_strategy="epoch",           # Save checkpoint every epoch
    save_total_limit=3,              # Only keep the 3 most recent models
    load_best_model_at_end=True,     # Load the best model at the end
    metric_for_best_model="accuracy",# Metric to determine the best model
    greater_is_better=True,          # Whether the metric should be maximized or minimized
    lr_scheduler_type="linear",      # Type of learning rate scheduler
    warmup_steps=500,                # Warmup steps for learning rate scheduler
    gradient_accumulation_steps=2,   # Accumulate gradients over 2 steps
    fp16=True,                       # Enable mixed precision training
)


/home/sivakorn/forest_type/img_reg/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
metric = load_metric("seqeval")

/tmp/ipykernel_24949/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [24]:
tag_id

{'Appearance': 0,
 'DeliveryTime': 1,
 'Price': 2,
 'Quality': 3,
 'StoreService': 4,
 'NEG-Quality': 5,
 'O': 6,
 'NEG-Appearance': 7,
 'NEG-StoreService': 8,
 'Size': 9,
 'NEG-DeliveryTime': 10,
 'Packaging': 11,
 'NEG-Price': 12,
 'NEG-Size': 13,
 'NEG-Packaging': 14}

In [25]:
def convert_to_bio(tags):
    bio_tags = []
    for tag in tags:
        if tag == 'O':
            bio_tags.append(tag)
        else:
            bio_tags.append(f'B-{tag}')
    return bio_tags

# Example usage
custom_tags = ['Appearance', 'DeliveryTime', 'Price', 'Quality', 'StoreService', 'NEG-Quality', 'O', 
               'NEG-Appearance', 'NEG-StoreService', 'Size', 'NEG-DeliveryTime', 'Packaging', 
               'NEG-Price', 'NEG-Size', 'NEG-Packaging']
bio_tags = convert_to_bio(custom_tags)
print(bio_tags)


['B-Appearance', 'B-DeliveryTime', 'B-Price', 'B-Quality', 'B-StoreService', 'B-NEG-Quality', 'O', 'B-NEG-Appearance', 'B-NEG-StoreService', 'B-Size', 'B-NEG-DeliveryTime', 'B-Packaging', 'B-NEG-Price', 'B-NEG-Size', 'B-NEG-Packaging']


In [26]:
id_tag = {value: key for key, value in tag_id.items()}
id_tag

{0: 'Appearance',
 1: 'DeliveryTime',
 2: 'Price',
 3: 'Quality',
 4: 'StoreService',
 5: 'NEG-Quality',
 6: 'O',
 7: 'NEG-Appearance',
 8: 'NEG-StoreService',
 9: 'Size',
 10: 'NEG-DeliveryTime',
 11: 'Packaging',
 12: 'NEG-Price',
 13: 'NEG-Size',
 14: 'NEG-Packaging'}

In [27]:
id_tag  = {
    0: 'B-Appearance',
    1: 'B-DeliveryTime',
    2: 'B-Price',
    3: 'B-Quality',
    4: 'B-StoreService',
    5: 'B-NEG-Quality',
    6: 'O',
    7: 'B-NEG-Appearance',
    8: 'B-NEG-StoreService',
    9: 'B-Size',
    10: 'B-NEG-DeliveryTime',
    11: 'B-Packaging',
    12: 'B-NEG-Price',
    13: 'B-NEG-Size',
    14: 'B-NEG-Packaging'
}


In [28]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id_tag[p].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_tag[l].replace('_','-') for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels,zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.723146,0.042254,0.051584,0.046455,0.066050
2,No log,2.712165,0.045178,0.053846,0.049133,0.075297
3,No log,2.693686,0.048976,0.055204,0.051904,0.090489
4,No log,2.668365,0.055605,0.056561,0.056079,0.119881
5,No log,2.636221,0.067181,0.055204,0.060606,0.150264


In [ ]:
trainer.evaluate()

{'eval_loss': 3.353506088256836,
 'eval_precision': 0.6347093970668115,
 'eval_recall': 0.6171111697913916,
 'eval_f1': 0.6257865845494712,
 'eval_accuracy': 0.6064638783269962,
 'eval_runtime': 0.2821,
 'eval_samples_per_second': 216.247,
 'eval_steps_per_second': 7.09,
 'epoch': 500.0}

## Prediction

In [ ]:
import pandas as pd
test = pd.read_csv("/home/sivakorn/Typhoon/aspect-based/test.csv")

In [ ]:
test[:10]

,id,input
0,1_1,สินค้า
1,1_2,มี
2,1_3,การ
3,1_4,การกระแทก
4,1_5,อย่าง
5,1_6,ดี
6,1_7,มี
7,1_8,การ
8,1_9,ห่อ
9,1_10,บับเบิ้ล


In [ ]:
import csv
from collections import defaultdict
from datasets import Dataset

def process_csv(file_path):
    word_groups = defaultdict(list)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            if row and len(row) == 2:
                id_prefix, word = row
                id_num = id_prefix.split('_')[0]
                word_groups[id_num].append(word)

    dataset = [{"id": k, "tokens": v} for k, v in word_groups.items()]
    return Dataset.from_list(dataset)

# Usage
file_path = '/home/sivakorn/Typhoon/aspect-based/test.csv'
huggingface_dataset = process_csv(file_path)
print(huggingface_dataset)

Dataset({
    features: ['id', 'tokens'],
    num_rows: 153
})


In [ ]:
def tokenize_for_prediction(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True, return_tensors="pt")
    
    word_ids_list = []
    for i in range(len(examples["tokens"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        word_ids_list.append(word_ids)
    
    tokenized_inputs["word_ids"] = word_ids_list
    
    return tokenized_inputs

# Apply the tokenization to your dataset
tokenized_dataset = huggingface_dataset.map(tokenize_for_prediction, batched=True)

# Remove the original 'tokens' column as it's no longer needed

Map: 100%|██████████| 153/153 [00:00<00:00, 2048.14 examples/s]


In [ ]:
tokenized_dataset

Dataset({
    features: ['id', 'tokens', 'input_ids', 'attention_mask', 'word_ids'],
    num_rows: 153
})

In [ ]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
predictions, _, _ = trainer.predict(tokenized_dataset)
predictions = np.argmax(predictions, axis=2)

In [ ]:
tag_id = {0: 'Appearance',
 1: 'DeliveryTime',
 2: 'Price',
 3: 'Quality',
 4: 'StoreService',
 5: 'NEG-Quality',
 6: 'O',
 7: 'NEG-Appearance',
 8: 'NEG-StoreService',
 9: 'Size',
 10: 'NEG-DeliveryTime',
 11: 'Packaging',
 12: 'NEG-Price',
 13: 'NEG-Size',
 14: 'NEG-Packaging'}

In [ ]:
len(tokenized_dataset[0]['tokens'])

36

In [ ]:
predictions[0]

array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11])

In [ ]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
tokenized_dataset

Dataset({
    features: ['id', 'tokens', 'input_ids', 'attention_mask', 'word_ids'],
    num_rows: 153
})

In [ ]:
import numpy as np
import pandas as pd


# Perform predictions
predictions, labels, _ = trainer.predict(tokenized_dataset)
predictions = np.argmax(predictions, axis=2)

# Function to find the most frequent element in a list
def most_frequent(lst):
    return max(set(lst), key=lst.count)

# Prepare predictions and labels
# Prepare predictions
_predictions = []

for i in range(len(tokenized_dataset['tokens'])):
    p = []
    wid = tokenized_dataset['word_ids'][i]
    for j in range(len(tokenized_dataset['tokens'][i])):
        if j in wid:
            start_index = wid.index(j)
            count_j = wid.count(j)
            p.append(most_frequent(list(predictions[i][start_index:start_index + count_j])))
        else:
            # Handle case where j is not in wid, e.g., by appending a default value or skipping
            p.append(0)  # Appending 0 as default, modify as needed
    _predictions.append(p)

In [ ]:
len(tokenized_dataset[1]['tokens'])

34

In [ ]:
len(_predictions[1])

34

In [ ]:
true_predictions = [
    [tag_id[p] for p in prediction]
    for prediction in _predictions
]


In [ ]:
output_list = []

for i, prediction in enumerate(true_predictions, start=1):
    for j, pred in enumerate(prediction, start=1):
        output_list.append({
            "id": f"{i}_{j}",
            "output": pred
        })

# Convert to DataFrame
df = pd.DataFrame(output_list)

# Write to CSV
df.to_csv("HoogBERTa-NER-lst202.csv", index=False)

# Print the DataFrame (optional)
print(df)

          id       output
0        1_1    Packaging
1        1_2    Packaging
2        1_3    Packaging
3        1_4    Packaging
4        1_5    Packaging
...      ...          ...
6387  153_31  NEG-Quality
6388  153_32  NEG-Quality
6389  153_33  NEG-Quality
6390  153_34  NEG-Quality
6391  153_35  NEG-Quality

[6392 rows x 2 columns]


Ref : https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb